<a href="https://colab.research.google.com/github/LalithShankar/BERT-for-Sentiment-Analysis/blob/master/Topic%20extraction%20Part-2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import shorttext
import numpy as np
import pandas as pd
import re

In [20]:
!pip install stem

     |████████████████████████████████| 2.9MB 5.0MB/s 
  Created wheel for stem: filename=stem-1.8.0-cp36-none-any.whl size=436038 sha256=a34b08b039b9726c1e2bef35155730d63626e8d8b5e73a4410183cdc4352d188
  Stored in directory: /root/.cache/pip/wheels/02/3a/ee/1094b166e029353f892c0b121aa02f48aff5e658396924bc2a
Successfully built stem


In [0]:
from stemming.porter import stem
 

In [23]:
pipeline = [lambda s: re.sub('[^\w\s]', '', s),
            lambda s: re.sub('[\d]', '', s),
            lambda s: s.lower(),
            
 ]
txtpreproceesor = shorttext.utils.text_preprocessor(pipeline)
txtpreproceesor

<function shorttext.utils.textpreprocessing.text_preprocessor.<locals>.<lambda>>

In [24]:
docx = pd.read_csv('Rule_Based_Sentiment.csv')
docx

,index,Data,Polarity,Subjectivity
0,0,ADMINISTRATIVE PROCEEDING \nFile No. 3 - 18279...,-0.083333,0.400000
1,1,To help detect potential violations of the sec...,-0.041667,0.458333
2,2,"According to the SEC’s order, starting in appr...",-0.068449,0.553209
3,3,The SEC’s order instituting a settled administ...,-0.100000,0.600000
4,4,The SEC’s investigation was conducted by Richa...,-0.100000,0.600000
5,5,The SEC appreciates the assistance of FinCEN a...,-0.050000,0.300000


In [0]:
docids = list(docx['index'])
corpus = [txtpreproceesor(Data).split(' ') for Data in docx['Data']]
# corpus = [txtpreproceesor(Data).split(' ') for Data in usprezdf['Data']]
corpus

In [26]:
dtm = shorttext.utils.DocumentTermMatrix(corpus, docids=docids, tfidf=False)
print(dtm)

In [0]:
dtm.get_doc_tokens('paul')

In [0]:
dtm.get_doc_frequency(stem('change'))

In [0]:
pip install gensimpip install gensim

In [0]:
from gensim import matutils, models
import scipy.sparse

In [0]:
tdm = docx.transpose()
tdm.head()

In [0]:
from nltk import word_tokenize, pos_tag

def nouns(text):
    '''Given a string of text, tokenize the text and pull out only the nouns.'''
    is_noun = lambda pos: pos[:2] == 'NN'
    tokenized = word_tokenize(text)
    all_nouns = [word for (word, pos) in pos_tag(tokenized) if is_noun(pos)] 
    return ' '.join(all_nouns)

In [32]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [33]:

data_nouns = pd.DataFrame(docx.Data.apply(nouns))
data_nouns

,Data
0,PROCEEDING File No SEC Charges Wells Fargo Adv...
1,violations securities laws money violations Ba...
2,SEC ’ s order March management Wells Fargo Adv...
3,SEC ’ order proceeding Wells Fargo Advisors Se...
4,SEC ’ investigation Richard G. Stoltz Paul Men...
5,SEC assistance FinCEN Financial Industry Regul...


In [0]:
# Create a new document-term matrix using only nouns
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer

# Re-add the additional stop words since we are recreating the document-term matrix
add_stop_words = ['like', 'im', 'know', 'just', 'dont', 'thats', 'right', 'people',
                  'youre', 'got', 'gonna', 'time', 'think', 'yeah', 'said']
stop_words = text.ENGLISH_STOP_WORDS.union(add_stop_words)
stop_words


In [0]:
# Recreate a document-term matrix with only nouns
cvn = CountVectorizer(stop_words=stop_words)
data_cvn = cvn.fit_transform(data_nouns.Data)
data_dtmn = pd.DataFrame(data_cvn.toarray(), columns=cvn.get_feature_names())
data_dtmn.index = data_nouns.index
data_dtmn[]

In [60]:
# Create the gensim corpus
corpusn = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmn.transpose()))
print(corpusn)


In [0]:
import pickle

In [0]:
cv = pickle.load(open("cv_stop.pkl", "rb"))

In [0]:
# Create the vocabulary dictionary
id2wordn = dict((v, k) for k, v in cvn.vocabulary_.items())
id2wordn

In [50]:
# Let's start with 2 topics
ldan = models.LdaModel(corpus=corpusn, num_topics=2, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.039*"fargo" + 0.039*"wells" + 0.039*"activity" + 0.039*"advisors" + 0.039*"sar" + 0.033*"sec" + 0.032*"sars" + 0.026*"order" + 0.018*"firm" + 0.018*"investigators"'),
 (1,
  '0.040*"activity" + 0.033*"sars" + 0.026*"broker" + 0.026*"fincen" + 0.026*"sar" + 0.026*"advisors" + 0.025*"wells" + 0.025*"fargo" + 0.018*"securities" + 0.018*"dealers"')]

In [51]:
# Let's try topics = 3
ldan = models.LdaModel(corpus=corpusn, num_topics=3, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.073*"activity" + 0.073*"sar" + 0.059*"sars" + 0.032*"fargo" + 0.032*"wells" + 0.032*"advisors" + 0.032*"investigators" + 0.032*"reviews" + 0.032*"firm" + 0.018*"order"'),
 (1,
  '0.035*"sars" + 0.035*"activity" + 0.034*"sar" + 0.024*"fincen" + 0.024*"broker" + 0.024*"bsa" + 0.024*"days" + 0.024*"violations" + 0.024*"dealers" + 0.024*"paul"'),
 (2,
  '0.059*"advisors" + 0.059*"fargo" + 0.059*"wells" + 0.040*"sec" + 0.031*"order" + 0.022*"securities" + 0.022*"charges" + 0.022*"activity" + 0.022*"exchange" + 0.022*"proceeding"')]

In [52]:
# Let's try 4 topics
ldan = models.LdaModel(corpus=corpusn, num_topics=4, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.058*"activity" + 0.058*"sar" + 0.047*"sars" + 0.025*"investigation" + 0.025*"mensheha" + 0.025*"paul" + 0.025*"sec" + 0.025*"reviews" + 0.025*"firm" + 0.025*"investigators"'),
 (1,
  '0.051*"activity" + 0.041*"sars" + 0.032*"sar" + 0.031*"wells" + 0.031*"advisors" + 0.031*"fargo" + 0.031*"broker" + 0.022*"fincen" + 0.022*"violations" + 0.022*"days"'),
 (2,
  '0.038*"sec" + 0.038*"order" + 0.037*"financial" + 0.037*"industry" + 0.037*"regulatory" + 0.037*"authority" + 0.037*"assistance" + 0.037*"fincen" + 0.009*"advisors" + 0.009*"wells"'),
 (3,
  '0.065*"fargo" + 0.065*"wells" + 0.065*"advisors" + 0.045*"sec" + 0.045*"order" + 0.025*"proceeding" + 0.025*"securities" + 0.025*"exchange" + 0.025*"act" + 0.025*"section"')]

In [0]:
# Let's create a function to pull out nouns from a string of text
def nouns_adj(text):
    '''Given a string of text, tokenize the text and pull out only the nouns and adjectives.'''
    is_noun_adj = lambda pos: pos[:2] == 'NN' or pos[:2] == 'JJ'
    tokenized = word_tokenize(text)
    nouns_adj = [word for (word, pos) in pos_tag(tokenized) if is_noun_adj(pos)] 
    return ' '.join(nouns_adj)


In [0]:
# Apply the nouns function to the transcripts to filter only on nouns
data_nouns_adj = pd.DataFrame(docx.Data.apply(nouns_adj))
data_nouns_adj

In [0]:
# Create a new document-term matrix using only nouns and adjectives, also remove common words with max_df
cvna = CountVectorizer(stop_words=stop_words, max_df=.8)
data_cvna = cvna.fit_transform(data_nouns_adj.Data)
data_dtmna = pd.DataFrame(data_cvna.toarray(), columns=cvna.get_feature_names())
data_dtmna.index = data_nouns_adj.index
data_dtmna

In [0]:
# Create the gensim corpus
corpusna = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmna.transpose()))

# Create the vocabulary dictionary
id2wordna = dict((v, k) for k, v in cvna.vocabulary_.items())

In [65]:
# Let's start with 2 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=2, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.041*"advisors" + 0.041*"fargo" + 0.041*"wells" + 0.036*"activity" + 0.026*"sar" + 0.026*"sars" + 0.017*"order" + 0.012*"cease" + 0.012*"desist" + 0.012*"exchange"'),
 (1,
  '0.031*"sars" + 0.031*"activity" + 0.031*"sar" + 0.031*"broker" + 0.030*"fincen" + 0.022*"suspicious" + 0.022*"previously" + 0.022*"dealers" + 0.022*"filing" + 0.022*"violations"')]

In [0]:
# Let's try 3 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=3, id2word=id2wordna, passes=10)
ldana.print_topics()

In [0]:
# Let's try 4 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=4, id2word=id2wordna, passes=10)
ldana.print_topics()

In [84]:
# Our final LDA model (for now)
ldana = models.LdaModel(corpus=corpusna, num_topics=4, id2word=id2wordna, passes=80)
ldana.print_topics()

[(0,
  '0.042*"broker" + 0.042*"sars" + 0.042*"sar" + 0.042*"activity" + 0.029*"fincen" + 0.029*"suspicious" + 0.029*"dealers" + 0.029*"days" + 0.029*"previously" + 0.029*"bsa"'),
 (1,
  '0.037*"paul" + 0.037*"mensheha" + 0.037*"investigation" + 0.021*"hoffmeyer" + 0.021*"george" + 0.021*"stoltz" + 0.021*"chicago" + 0.021*"regional" + 0.021*"carlson" + 0.021*"spicher"'),
 (2,
  '0.008*"order" + 0.008*"regulatory" + 0.008*"financial" + 0.008*"fincen" + 0.008*"assistance" + 0.008*"industry" + 0.008*"authority" + 0.008*"act" + 0.008*"administrative" + 0.008*"securities"'),
 (3,
  '0.054*"fargo" + 0.054*"wells" + 0.054*"advisors" + 0.047*"activity" + 0.034*"sar" + 0.034*"sars" + 0.021*"order" + 0.015*"investigators" + 0.015*"firm" + 0.015*"reviews"')]

In [89]:

# Let's take a look at which topics each transcript contains
corpus_transformed = ldana[corpusna]
d = list(corpus_transformed,data_dtmna.index)

TypeError: ignored

In [88]:

# Let's take a look at which topics each transcript contains
corpus_transformed = ldana[corpusna]
d = list(zip([a for [(a, b)] in corpus_transformed], data_dtmna.index))

ValueError: ignored

In [78]:
d

[[(4, 0.9818443)],
 [(3, 0.982969)],
 [(2, 0.8255362), (4, 0.1605595)],
 [(4, 0.97473747)],
 [(0, 0.9666658)],
 [(0, 0.020833686),
  (1, 0.020834122),
  (2, 0.020848554),
  (3, 0.020877931),
  (4, 0.020865232),
  (5, 0.89574045)]]

In [0]:
cor=list(corpus_transformed)
cor.append(data_dtmna.index)

[[(3, 0.98348296)],
 [(0, 0.9845643)],
 [(3, 0.98422164)],
 [(3, 0.9771525)],
 [(0, 0.010007286), (1, 0.96996963), (2, 0.010019406), (3, 0.010003668)],
 [(0, 0.032031048), (1, 0.9048761), (2, 0.031322204), (3, 0.031770635)],
    11  17a  accounts  act  ...  transactions  treasury  violations  wells
 0   0    0         1    0  ...             0         0           0      3
 1   0    0         0    1  ...             1         1           2      0
 2   1    0         0    0  ...             0         0           0      2
 3   0    1         0    1  ...             0         0           0      3
 4   0    0         0    0  ...             0         0           0      0
 5   0    0         0    0  ...             0         0           0      0
 
 [6 rows x 119 columns]]

In [91]:
cor

[[(3, 0.9834832)],
 [(0, 0.98456216)],
 [(3, 0.9842215)],
 [(3, 0.9771524)],
 [(0, 0.010007286), (1, 0.96996963), (2, 0.010019401), (3, 0.010003668)],
 [(0, 0.0320511), (1, 0.9048536), (2, 0.031322222), (3, 0.031773098)],
 RangeIndex(start=0, stop=6, step=1)]